In [34]:
import os
import json
#import tqdm
import re
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
#import spacy
import scipy
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from gensim import corpora, models, similarities
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [35]:
df_ori = pd.read_csv('./data/data job posts.csv')
df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['jobpost','Title'])
print(df.shape)
print("Removed {0} duplicates (based on jobpost + Title)".format(df_ori.shape[0]-df.shape[0]))

(19001, 24)
(18892, 24)
Removed 109 duplicates (based on jobpost + Title)


In [36]:
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,Location,JobDescription,JobRequirment,RequiredQual,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\r\nindivi...",NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,"IREX Armenia Main Office; Yerevan, Armenia \r\...",NaN,NaN,- Bachelor's Degree; Master's is preferred;\r\...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,"Yerevan, Armenia",Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,"Manila, Philippines",The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",NaN,- Rendering technical assistance to Database M...,- University degree; economical background is ...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [37]:
df["RequiredQual"] = df['RequiredQual'].astype(str)

In [38]:
df['RequiredQual_token'] = df['RequiredQual'].map(word_tokenize)

In [39]:
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [40]:
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item for item in x if item not in stop_words])
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item for item in x if item not in string.punctuation])

In [41]:
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item.lower() for item in x])

In [42]:
df['RequiredQual_token'].head()

0    [to, perform, job, successfully, individual, m...
1    [bachelor, 's, degree, master, 's, preferred, ...
2    [degree, environmentally, related, field, 5, y...
3    [advanced, degree, public, health, social, sci...
4    [university, degree, economical, background, p...
Name: RequiredQual_token, dtype: object

In [43]:
#Making dictionary out of words
dictionary = corpora.Dictionary(df['RequiredQual_token'])
dictionary.save('/job_req.dict')

In [44]:
 corpus = [dictionary.doc2bow(text) for text in df['RequiredQual_token']]

In [45]:
corpora.MmCorpus.serialize('/job_req.mm', corpus)

In [46]:
corpus = corpora.MmCorpus('/job_req.mm')
print(corpus)

MmCorpus(18892 documents, 14496 features, 826035 non-zero entries)


In [47]:
#define a 2-dimensional LSI space:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [48]:
doc = "Which jobs require female candidates?"

In [49]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.00048706500713036706), (1, 0.00034981003278805633)]


In [50]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [51]:
index.save('/job_req.index')
index = similarities.MatrixSimilarity.load('/job_req.index')

In [52]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims)))

[(0, 0.93575835), (1, 0.9736101), (2, 0.7315645), (3, 0.74041146), (4, 0.30829814), (5, 0.8180501), (6, 0.8468059), (7, -0.5864544), (8, 0.73570347), (9, 0.8472518), (10, -0.5864544), (11, -0.5864544), (12, 0.85214347), (13, 0.9432759), (14, 0.7446501), (15, 0.2828247), (16, -0.5864544), (17, 0.7949203), (18, 0.7480471), (19, 0.575567), (20, 0.898033), (21, 0.3602091), (22, 0.89678377), (23, 0.9843256), (24, 0.8023716), (25, 0.992769), (26, -0.5864544), (27, 0.8023716), (28, -0.5864544), (29, 0.8407926), (30, -0.5864544), (31, 0.6085226), (32, 0.8465152), (33, 0.24610707), (34, 0.9118847), (35, 0.6646267), (36, 0.73074555), (37, 0.7902369), (38, -0.5864544), (39, 0.95888317), (40, 0.7959434), (41, 0.7353065), (42, 0.56195384), (43, 0.9364363), (44, 0.35334757), (45, 0.8742079), (46, 0.9706421), (47, 0.74548), (48, 0.7599921), (49, 0.9908353), (50, 0.96477634), (51, -0.5864544), (52, 0.79820186), (53, 0.9694228), (54, 0.9677145), (55, 0.89974356), (56, 0.64548564), (57, 0.75861144), (58

In [53]:
sims = sorted(list(enumerate(sims)), key=lambda item: -item[1])
print(sims[:10])

[(6894, 1.0), (8811, 1.0), (9734, 1.0), (7876, 0.99999994), (11252, 0.99999994), (10005, 0.9999999), (11784, 0.9999999), (13728, 0.99999964), (8652, 0.9999994), (11030, 0.9999994)]


In [58]:
for index,similarity in enumerate(sims[:10]):
    print(df['RequiredQual'].loc[[index]]+ "index" +str(index))